In [6]:
library(TCGAbiolinks)
library(SummarizedExperiment)
library(edgeR)

In [7]:
# list of hotspot and wild-type aliquot ids in brca and coad with experession data
listsamples_br <- c("TCGA-A7-A13E-01A","TCGA-A7-A0CH-01A","TCGA-A2-A0CX-01A","TCGA-C8-A137-01A","TCGA-AR-A0TP-01A","TCGA-AO-A12F-01A","TCGA-A2-A0SW-01A","TCGA-BH-A0DV-01A","TCGA-C8-A12O-01A","TCGA-C8-A12M-01A","TCGA-A2-A0T5-01A")
listsamples_cd <- c("TCGA-4N-A93T-01A","TCGA-AA-A01X-01A","TCGA-NH-A6GA-01A","TCGA-A6-A567-01A","TCGA-AD-6889-01A","TCGA-AA-A01P-01A","TCGA-AA-A022-01A","TCGA-NH-A8F8-01A","TCGA-QG-A5YV-01A","TCGA-QL-A97D-01A","TCGA-DM-A28G-01A","TCGA-AY-A71X-01A")

# query the file for primary tumors

query <- GDCquery(project = "TCGA-BRCA",
                  data.category = "Gene expression",
                  data.type = "Gene expression quantification",
                  experimental.strategy = "RNA-Seq",
                  platform = "Illumina HiSeq",
                  file.type = "results",
                  barcode = listsamples_br, 
                  legacy = TRUE)

query2 <- GDCquery(project = "TCGA-COAD",
                  data.category = "Gene expression",
                  data.type = "Gene expression quantification",
                  experimental.strategy = "RNA-Seq",
                  platform = "Illumina HiSeq",
                  file.type = "results",
                  barcode = listsamples_cd, 
                  legacy = TRUE)
# Download the data
GDCdownload(query)
GDCdownload(query2)

####BRCA####

# Prepare expression matrix with geneID in the rows and samples (barcode) in the columns
# rsem.genes.results as values

BRCARnaseqSE <- GDCprepare(query)

BRCAMatrix <- assay(BRCARnaseqSE,"raw_count") # or BRCAMatrix <- assay(BRCARnaseqSE,"raw_count")



--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg19

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-BRCA

--------------------

oo Filtering results

--------------------

ooo By platform

ooo By experimental.strategy

ooo By data.type

ooo By file.type

ooo By barcode

----------------

oo Checking data

----------------

ooo Check if there are duplicated cases

ooo Check if there results for the query

-------------------

o Preparing output

-------------------

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg19

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-COAD

--------------------

Downloading: 6.7 MB       


GDCdownload will download 11 files. A total of 16.59348 MB




Downloading: 6.6 MB     

-------------------

oo Reading 11 files

-------------------



|====================================================|100%                      Completed after 0 s 


-------------------

oo Merging 11 files

-------------------

Accessing grch37.ensembl.org to get gene information


Cache found

Starting to add information to samples

 => Add clinical information to samples

 => Adding TCGA molecular information from marker papers

 => Information will have prefix 'paper_' 

brca subtype information from:doi.org/10.1016/j.ccell.2018.03.014



In [8]:
# For gene expression if you need to see a boxplot correlation and AAIC plot to define outliers you can run
BRCARnaseq_CorOutliers <- TCGAanalyze_Preprocessing(BRCARnaseqSE)


dataNorm <- TCGAanalyze_Normalization(tabDF = BRCARnaseq_CorOutliers, geneInfo =  geneInfo)

dataFilt <- TCGAanalyze_Filtering(tabDF = dataNorm,
                                  method = "quantile", 
                                  qnt.cut =  0.25)

ht=c("TCGA-A7-A13E-01A-11R-A12P-07","TCGA-A2-A0CX-01A-21R-A00Z-07","TCGA-C8-A137-01A-11R-A115-07", "TCGA-AR-A0TP-01A-11R-A084-07","TCGA-A2-A0SW-01A-11R-A084-07",  "TCGA-C8-A12O-01A-11R-A115-07")

wt=c("TCGA-A7-A0CH-01A-21R-A00Z-07","TCGA-C8-A12M-01A-11R-A115-07","TCGA-AO-A12F-01A-11R-A115-07","TCGA-BH-A0DV-01A-21R-A12P-07","TCGA-A2-A0T5-01A-21R-A084-07")


dataDEGs <- TCGAanalyze_DEA(mat1 = dataFilt[,wt],
                            mat2 = dataFilt[,ht],
                            Cond1type = "Wild",
                            Cond2type = "Hotspot",
                            method = "glmLRT")

dataDEGsFiltLevel <- TCGAanalyze_LevelTab(dataDEGs,"Hotspot","Wild",
                                          dataFilt[,ht],dataFilt[,wt])

write.table(dataDEGsFiltLevel, file="Results/BRCA_DEG_HW_raw.tsv", sep="\t")

#### COAD####

COADRnaseqSE <- GDCprepare(query2)
COADMatrix <- assay(COADRnaseqSE,"raw_count") # or COADMatrix <- assay(COADRnaseqSE,"raw_count")

# For gene expression if you need to see a boxplot correlation and AAIC plot to define outliers you can run
COADRnaseq_CorOutliers <- TCGAanalyze_Preprocessing(COADRnaseqSE)


dataNorm <- TCGAanalyze_Normalization(tabDF = COADRnaseq_CorOutliers, geneInfo =  geneInfo)

dataFilt <- TCGAanalyze_Filtering(tabDF = dataNorm,
                                  method = "quantile", 
                                  qnt.cut =  0.25)

ht=c("TCGA-4N-A93T-01A-11R-A37K-07", "TCGA-NH-A6GA-01A-11R-A37K-07","TCGA-A6-A567-01A-31R-A28H-07" , "TCGA-NH-A8F8-01A-72R-A41B-07" ,"TCGA-QG-A5YV-01A-11R-A28H-07" ,"TCGA-QL-A97D-01A-12R-A41B-07")
wt=c("TCGA-AA-A01X-01A-21R-A083-07","TCGA-AD-6889-01A-11R-1928-07","TCGA-AA-A01P-01A-21R-A083-07" ,"TCGA-DM-A28G-01A-11R-A16W-07" ,"TCGA-AY-A71X-01A-12R-A37K-07")

dataDEGs <- TCGAanalyze_DEA(mat1 = dataFilt[,wt],
                            mat2 = dataFilt[,ht],
                            Cond1type = "Wild",
                            Cond2type = "Hotspot",
                            method = "glmLRT")

dataDEGsFiltLevel <- TCGAanalyze_LevelTab(dataDEGs,"Hotspot","Wild",
                                          dataFilt[,ht],dataFilt[,wt])

write.table(dataDEGsFiltLevel, file="Results/COAD_DEG_HW_raw.tsv", sep="\t")


I Need about  2.7 seconds for this Complete Normalization Upper Quantile  [Processing 80k elements /s]  

Step 1 of 4: newSeqExpressionSet ...

Step 2 of 4: withinLaneNormalization ...

Step 3 of 4: betweenLaneNormalization ...

Step 4 of 4: exprs ...

Batch correction skipped since no factors provided

----------------------- DEA -------------------------------

there are Cond1 type Wild in  5 samples

there are Cond2 type Hotspot in  6 samples

there are  14892 features as miRNA or genes 

I Need about  5.5 seconds for this DEA. [Processing 30k elements /s]  

----------------------- END DEA -------------------------------

-------------------

oo Reading 11 files

-------------------



|====================================================|100%                      Completed after 0 s 


-------------------

oo Merging 11 files

-------------------

Accessing grch37.ensembl.org to get gene information


Cache found

Starting to add information to samples

 => Add clinical information to samples

 => Adding TCGA molecular information from marker papers

 => Information will have prefix 'paper_' 

coad subtype information from:doi:10.1038/nature11252

I Need about  2.7 seconds for this Complete Normalization Upper Quantile  [Processing 80k elements /s]  

Step 1 of 4: newSeqExpressionSet ...

Step 2 of 4: withinLaneNormalization ...

Step 3 of 4: betweenLaneNormalization ...

Step 4 of 4: exprs ...

Batch correction skipped since no factors provided

----------------------- DEA -------------------------------

there are Cond1 type Wild in  5 samples

there are Cond2 type Hotspot in  6 samples

there are  14890 features as miRNA or genes 

I Need about  5.5 seconds for this DEA. [Processing 30k elements /s]  

----------------------- END DEA ------------------------------